In [ ]:
from pyspark.sql.functions import col, when, isnull
from pyspark.sql.types import IntegerType

df = spark.read.format("delta").load("Tables/solicitudes")

# Transformar solicitud_credito a estado
df = df.withColumn("estado_solicitud",
                   when(col("solicitud_credito") == 1, "Aprobado")
                   .when(col("solicitud_credito") == 0, "Rechazado")
                   .when(col("solicitud_credito").isNull(), "Pendiente")
                   .otherwise("Pendiente"))

# Manejar valores faltantes
df = df.fillna({"ingresos_anuales": 0, "deuda_actual": 0, "puntaje_crediticio": 0})

# Convertir puntaje_crediticio a entero
df = df.withColumn("puntaje_crediticio", col("puntaje_crediticio").cast(IntegerType()))

# Codificar columnas categóricas
df = df.withColumn("historial_pagos_encoded",
                   when(col("historial_pagos") == "Bueno", 2)
                   .when(col("historial_pagos") == "Regular", 1)
                   .otherwise(0))

df = df.withColumn("estado_civil_encoded",
                   when(col("estado_civil") == "Casado", 0)
                   .when(col("estado_civil") == "Soltero", 1)
                   .when(col("estado_civil") == "Viudo", 2)
                   .otherwise(3))
                   
df = df.withColumn("tipo_empleo_encoded",
                   when(col("tipo_empleo") == "Fijo", 0)
                   .when(col("tipo_empleo") == "Autonomo", 1)
                   .when(col("tipo_empleo") == "Desempleado", 2)
                   .otherwise(3))



# Guardar datos procesados
df.write.mode("overwrite").format("delta").save("Tables/solicitudes_processed")


StatementMeta(, 0f93f0a1-f030-480c-aabd-a980a14a1c9d, 3, Finished, Available, Finished)